In [1]:
### Load necessary libraries ###
import glob
import os
import librosa
import librosa.display
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix

import tensorflow as tf
from tensorflow import keras

# Global Configuration variables

In [16]:
# Debug and test variables
REDUCED_MODE = 1  # Do not process all the audio in the dataset (for faster execution during script development)
REDUCED_MODE_AUDIONUM = 100 # Number of dataset audios per folder to process in the reduced-mode.

# Audio processing variables
AUDIO_SR = 44100 # Audio sampling in Hertz
AUDIO_N_FFT = 2048 # Samples per FFT window
AUDIO_WINSIZE = 1024
AUDIO_HOP_LEN = 512
AUDIO_N_MELS = 60
AUDIOSEG_SIZE = 41 # windows per subAudio / segment


# Method for feature extraction

In [18]:
# Divide an audio file into shorter parts.
def audioSegmentation(data, window_size):
    start = 0
    while start < len(data):
        yield int(start), int(start + window_size)
        start += (window_size // 2)     


In [19]:
### Define helper functions ###
def extractSoundFeature(audiosPath,audiosSubPaths,audiosExtension="*.wav", n_mels=AUDIO_N_MELS, n_windows=AUDIOSEG_SIZE, hopLength = AUDIO_HOP_LEN):
    
    subAudioSize = hopLength * (n_windows - 1)
    features, classes = [], []
    
    # Map all audio samples paths inside the folder
    allAudiosPath = glob.glob(os.path.join(audiosPath, audiosSubPath, audiosExtension));

    # Iterate and extract features from each audio
    idx = 0
    for audioPath in allAudiosPath:
        print('Processing sample: ', idx)
        idx=idx+1;
        
        subAudioLogSpect = []
        subAudioClass = []

        # Extract '.wav' audio to an 1-D array
        audioData, sr = librosa.load(audioPath, sr = AUDIO_SR)

        # Extract audio classification from file name
        audioClass = int(audioPath.split('/')[2].split('-')[1])

        # Loop and extract all audio segments
        for (start,end) in audioSegmentation(audioData, subAudioSize):
            
            if(len(audioData[start:end]) == subAudioSize):
                signal = audioData[start:end]
                melspec = librosa.feature.melspectrogram(y=signal,n_mels=n_mels)
                print('melspec shape: ', np.shape(melspec))


                logspec = librosa.amplitude_to_db(melspec)
                print('logspec shape: ', np.shape(logspec))


                logspec = logspec.T.flatten()[:, np.newaxis].T
                print('logspec aft flatten shape: ', np.shape(logspec))


                subAudioLogSpect.append(logspec)
                print('subAudioLogSpect shape: ', np.shape(subAudioLogSpect))
                subAudioClass.append(audioClass)
            
        subAudioLogSpect = np.asarray(subAudioLogSpect).reshape(len(subAudioLogSpect),n_mels,n_windows,1)
        print('subAudioLogSpect shape final : ', np.shape(subAudioLogSpect))    

        segment_features = np.concatenate( (subAudioLogSpect, np.zeros(np.shape(subAudioLogSpect))), axis=3 )
        print('segment_features shape final: ', np.shape(segment_features)) # add an array of zeros in the last dimension

            
        for i in range(len(segment_features)): 
            segment_features[i, :, :, 1] = librosa.feature.delta(segment_features[i, :, :, 0]) #Calculate delta / derivative of axis 1
        
        if len(segment_features) > 0: # if not empty, concatenate in features / classes array
            features.append(segment_features)
            classes.append(subAudioClass)
    
        print('features shape final: ', np.shape(features))
    
    return features, classes

In [20]:
# Pre-process and extract feature from the data
audiosPath = 'UrbanSounds8K/audio/'
save_dir = "UrbanSounds8K/processed/"
folds = audiosSubPaths = np.array(['fold1','fold2','fold3','fold4', 'fold5','fold6','fold7','fold8', 'fold9','fold10'])
for audiosSubPath in audiosSubPaths:
    features, labels = extractSoundFeature(audiosPath,audiosSubPath)
    #np.savez("{0}{1}".format(save_dir, audiosSubPath), features=features, labels=labels);

Processing sample:  0
melspec shape:  (60, 41)
logspec shape:  (60, 41)
logspec aft flatten shape:  (1, 2460)
subAudioLogSpect shape:  (1, 1, 2460)


In [4]:
### Define convolutional network architecture ###
def get_network():
    num_filters = [24,32,64,128] 
    pool_size = (2, 2) 
    kernel_size = (3, 3)  
    input_shape = (60, 41, 2)
    num_classes = 10
    keras.backend.clear_session()
    
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(24, kernel_size,
                padding="same", input_shape=input_shape))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.MaxPooling2D(pool_size=pool_size))

    model.add(keras.layers.Conv2D(32, kernel_size,
                                  padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))  
    model.add(keras.layers.MaxPooling2D(pool_size=pool_size))
    
    model.add(keras.layers.Conv2D(64, kernel_size,
                                  padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))  
    model.add(keras.layers.MaxPooling2D(pool_size=pool_size))
    
    model.add(keras.layers.Conv2D(128, kernel_size,
                                  padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))  

    model.add(keras.layers.GlobalMaxPooling2D())
    model.add(keras.layers.Dense(128, activation="relu"))
    model.add(keras.layers.Dense(num_classes, activation="softmax"))

    model.compile(optimizer=keras.optimizers.Adam(1e-4), 
        loss=keras.losses.SparseCategoricalCrossentropy(), 
        metrics=["accuracy"])
    return model

In [5]:
### Train and evaluate via 10-Folds cross-validation ###
accuracies = []
folds = np.array(['fold1','fold2','fold3','fold4',
                  'fold5','fold6','fold7','fold8',
                  'fold9','fold10'])
load_dir = "UrbanSounds8K/processed/"
kf = KFold(n_splits=10)

In [10]:
kf.split(folds)

<generator object _BaseKFold.split at 0x000001E5C8EAE180>

In [13]:
idx = 0

for train_index, test_index in kf.split(folds):
    idx= idx+1
    
    print('--> Starting a loop!')

    x_train, y_train = [], []

    # ---------------- STEP 1 ----------------
    # Load training data from 9 of 10 folders 
    # Loop through all training folders and gather data in single feature array
    for idxTrainFolder in train_index:
        print('-> Getting data from TRAINING folder ', idxTrainFolder+1)
        # Read pre-saved features or segments of an audio file (pre-processed)
        train_data = np.load("{0}/{1}.npz".format(load_dir,folds[idxTrainFolder]), allow_pickle=True)

        # Get the 'features' and 'labels' from current train folder
        features = np.concatenate(train_data["features"], axis=0) 
        labels = np.concatenate(train_data["labels"], axis=0)

        # Append all the 'features' and 'labels' train datasets 
        # in a single list containing all train folders data
        x_train.append(features)
        y_train.append(labels)

    # Merge all separate feature datasets in a single one (as if it was a single 'big' folder)
    x_train = np.concatenate(x_train, axis = 0).astype(np.float32)
    y_train = np.concatenate(y_train, axis = 0).astype(np.float32)
    
    # ---------------- STEP 2 ----------------
    # Load test data from 1 of 10 folders 
    # Load test data from the test folder
    print('-> Getting data from TESTING folder ', test_index+1)
    test_data = np.load("{0}/{1}.npz".format(load_dir, folds[test_index][0]), allow_pickle=True)
    x_test = test_data["features"]
    y_test = test_data["labels"]

    model = get_network()
    
    print('--> Fitting model!')
    
    model.fit(x_train, y_train, epochs = 5, batch_size = 24, verbose = 1)
    
    # evaluate on test set/fold
    y_true, y_pred = [], []
    for x, y in zip(x_test, y_test):
        # average predictions over segments of a sound clip
        avg_p = np.argmax(np.mean(model.predict(x), axis = 0))
        y_pred.append(avg_p) 
        # pick single label via np.unique for a sound clip
        y_true.append(np.unique(y)[0]) 
    
    accuracies.append(accuracy_score(y_true, y_pred))    

    if idx == 1:
        print('early break')
        break;

    print('Confusion matrix - loop: ', idx);
    allConfusionMatrix(idx) = confusion_matrix(y_pred,y_true)
    print(confusion_matrix)

print("Average 10 Folds Accuracy: {0}".format(np.mean(accuracies)))

--> Starting a loop!
-> Getting data from training folder  1
-> Getting data from training folder  2
-> Getting data from training folder  3
-> Getting data from training folder  4
-> Getting data from training folder  5
-> Getting data from training folder  6
-> Getting data from training folder  7
-> Getting data from training folder  8
-> Getting data from training folder  9
-> Getting data from test folder  [0]
--> Fitting model!
Epoch 1/5
2026/2026 [==============================] - 69s 34ms/step - loss: 1.1497 - accuracy: 0.6202
Epoch 2/5
2026/2026 [==============================] - 70s 35ms/step - loss: 0.6976 - accuracy: 0.7736
Epoch 3/5
2026/2026 [==============================] - 70s 34ms/step - loss: 0.5371 - accuracy: 0.8226
Epoch 4/5
2026/2026 [==============================] - 71s 35ms/step - loss: 0.4313 - accuracy: 0.8601
Epoch 5/5
2026/2026 [==============================] - 72s 35ms/step - loss: 0.3648 - accuracy: 0.8814
test
Average 10 Folds Accuracy: 0.5888917446323

In [14]:
allConfusionMatrix(idx) = confusion_matrix(y_pred,y_true)

SyntaxError: cannot assign to function call here. Maybe you meant '==' instead of '='? (2589643968.py, line 1)